In [1]:
import pandas as pd
import getschools as gs

In [1]:
def get_sheet(sheetname=0):
    ret = pd.read_excel("../raw/TESOL Bilingual Waivers educator counts.xlsx",sheetname=sheetname)

    ret.columns="district","year","educators"

    
    ret = gs.add_district_matches(ret)
    
    ret.columns="district","year","educators","district_code","check_dist_col"
    
    ret = gs.add_district_matches(ret)
    return ret
    
get_sheet().head()

NameError: global name 'pd' is not defined

In [3]:
bilingual=get_sheet(2)
bilingual.head()

,district,year,educators,district_code,check_dist_col,dist_code
0,Bridgeport Achievement First District,2010-11,1,2850013,Bridgeport Achievement First District,2850013
1,Bridgeport School District,2010-11,45,0150011,Bridgeport School District,0150011
2,Clemens College,2010-11,5,None,None,None
3,Connecticut College,2010-11,1,None,None,None
4,Connecticut Technical High School System,2010-11,9,9000016,Connecticut Technical High School System,9000016


In [4]:
tesol=get_sheet(3)
tesol.head()

,district,year,educators,district_code,check_dist_col,dist_code
0,Ansonia School District,2010-11,3,0020011,Ansonia School District,0020011
1,Area Cooperative Educational Services,2010-11,1,2440014,Area Cooperative Educational Services,2440014
2,Ashford School District,2010-11,1,0030011,Ashford School District,0030011
3,Avon School District,2010-11,2,0040011,Avon School District,0040011
4,Bais Binyomin Academy,2010-11,2,None,None,None


# Success rates

In [5]:
def dist_success(df):
    print "Success", len(df[df["district_code"].notnull()]["district"].unique())
    print "Failure", len(df[df["district_code"].isnull()]["district"].unique())
    
    print ""
    print ""
    print df[df["district_code"].isnull()]["district"].unique()

In [6]:
dist_success(bilingual)

Success 24
Failure 2


[u'Clemens College' u'Connecticut College']


In [7]:
dist_success(tesol)

Success 86
Failure 7


[u'Bais Binyomin Academy' u'Clemens College' u'Connecticut College'
 u'Hartford Seminary' u'Lyme Academy College of Fine Arts'
 u'Rensselaer at Hartford' u'Unified School District #1']


In [8]:
bilingual.head()

,district,year,educators,district_code,check_dist_col,dist_code
0,Bridgeport Achievement First District,2010-11,1,2850013,Bridgeport Achievement First District,2850013
1,Bridgeport School District,2010-11,45,0150011,Bridgeport School District,0150011
2,Clemens College,2010-11,5,None,None,None
3,Connecticut College,2010-11,1,None,None,None
4,Connecticut Technical High School System,2010-11,9,9000016,Connecticut Technical High School System,9000016


In [9]:
merged = bilingual.merge(tesol,how="outer",on=["district","year"],suffixes=("_bilingual","_tesol"))

merged.head()

,district,year,educators_bilingual,district_code_bilingual,check_dist_col_bilingual,dist_code_bilingual,educators_tesol,district_code_tesol,check_dist_col_tesol,dist_code_tesol
0,Bridgeport Achievement First District,2010-11,1.0,2850013,Bridgeport Achievement First District,2850013,NaN,NaN,NaN,NaN
1,Bridgeport School District,2010-11,45.0,0150011,Bridgeport School District,0150011,27.0,0150011,Bridgeport School District,0150011
2,Clemens College,2010-11,5.0,None,None,None,22.0,None,None,None
3,Connecticut College,2010-11,1.0,None,None,None,2.0,None,None,None
4,Connecticut Technical High School System,2010-11,9.0,9000016,Connecticut Technical High School System,9000016,21.0,9000016,Connecticut Technical High School System,9000016


In [10]:
# merged.columns = ["district","year",
#                   "educators_bilingual",
#                   "district_code",
#                   "check_dist_col_b",
#                   "educators_tesol",
#                   "district_code_b",
#                   "check_dist_col_b"]

merged["educators_bilingual"] = pd.to_numeric(merged["educators_bilingual"],errors="coerce").fillna(0)
merged["educators_tesol"] = pd.to_numeric(merged["educators_tesol"],errors="coerce").fillna(0)

merged["educators_total"] = merged["educators_bilingual"] + merged["educators_tesol"]
merged["educators_bilingual_pct"] = merged["educators_bilingual"] * 100 / merged["educators_total"]
merged["educators_tesol_pct"] = merged["educators_tesol"] * 100 / merged["educators_total"]

merged["district_code"] = merged["district_code_bilingual"].fillna(merged["district_code_tesol"])


merged.head()

,district,year,educators_bilingual,district_code_bilingual,check_dist_col_bilingual,dist_code_bilingual,educators_tesol,district_code_tesol,check_dist_col_tesol,dist_code_tesol,educators_total,educators_bilingual_pct,educators_tesol_pct,district_code
0,Bridgeport Achievement First District,2010-11,1.0,2850013,Bridgeport Achievement First District,2850013,0.0,NaN,NaN,NaN,1.0,100.000000,0.000000,2850013
1,Bridgeport School District,2010-11,45.0,0150011,Bridgeport School District,0150011,27.0,0150011,Bridgeport School District,0150011,72.0,62.500000,37.500000,0150011
2,Clemens College,2010-11,5.0,None,None,None,22.0,None,None,None,27.0,18.518519,81.481481,None
3,Connecticut College,2010-11,1.0,None,None,None,2.0,None,None,None,3.0,33.333333,66.666667,None
4,Connecticut Technical High School System,2010-11,9.0,9000016,Connecticut Technical High School System,9000016,21.0,9000016,Connecticut Technical High School System,9000016,30.0,30.000000,70.000000,9000016


In [11]:

pared_down = merged[["district","district_code","year","educators_total","educators_bilingual",
                     "educators_bilingual_pct","educators_tesol","educators_tesol_pct",
                    "check_dist_col_tesol"]]

pared_down.columns = ["district","district_code","year","educators_total","educators_bilingual",
                     "educators_bilingual_pct","educators_tesol","educators_tesol_pct",
                    "check_dist"]

pared_down["school_code"] = pared_down["district_code"]
pared_down.head()


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,district,district_code,year,educators_total,educators_bilingual,educators_bilingual_pct,educators_tesol,educators_tesol_pct,check_dist,school_code
0,Bridgeport Achievement First District,2850013,2010-11,1.0,1.0,100.000000,0.0,0.000000,NaN,2850013
1,Bridgeport School District,0150011,2010-11,72.0,45.0,62.500000,27.0,37.500000,Bridgeport School District,0150011
2,Clemens College,None,2010-11,27.0,5.0,18.518519,22.0,81.481481,None,None
3,Connecticut College,None,2010-11,3.0,1.0,33.333333,2.0,66.666667,None,None
4,Connecticut Technical High School System,9000016,2010-11,30.0,9.0,30.000000,21.0,70.000000,Connecticut Technical High School System,9000016


In [12]:
len(pared_down[pared_down["educators_tesol_pct"] > 75].index)

186

In [13]:
len(pared_down[pared_down["educators_bilingual_pct"] > 75].index)

5

In [14]:
pared_down.to_csv("../clean/csv/fte_bilingual_tesol.csv",index=False)